## DatetimeOutputParser

In [3]:
from langchain.output_parsers import DatetimeOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

output_parser = DatetimeOutputParser()
template = """Ansewer the users question:

{question}

{format_instructions}"""
prompt = PromptTemplate.from_template(
    template,
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)
prompt

PromptTemplate(input_variables=['question'], partial_variables={'format_instructions': "Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 0086-02-05T14:36:25.466397Z, 0454-03-10T01:25:38.791306Z, 0683-11-26T08:54:09.161213Z\n\nReturn ONLY this string, no other words!"}, template='Ansewer the users question:\n\n{question}\n\n{format_instructions}')

In [4]:
chain = prompt | ChatOpenAI(model="gpt-4o") | output_parser
output = chain.invoke({"question": "야구에서 삼성이 5번째로 우승한 연도는?"})
print(output)

2011-10-31 23:59:59.999999


## EnumOutputParser

In [15]:
from langchain.output_parsers.enum import EnumOutputParser
from enum import Enum

class Colors(Enum):
    RED = '빨간색'
    BLUE = '파란색'
    GREEN = '초록색'

enum_parser = EnumOutputParser(enum=Colors)

In [22]:
from langchain_core.prompts import PromptTemplate

enum_template =  """다음의 물체는 어떤 색깔인가요? 색깔로만 대답해주세요.
    
    Object: {object}
    
    Instructions: {format_instructions}"""
enum_prompt = PromptTemplate.from_template(
    template=enum_template,
    partial_variables={"format_instructions": enum_parser.get_format_instructions()}
)

enum_chain = enum_prompt | ChatOpenAI(model="gpt-4o") | enum_parser
response = enum_chain.invoke({"object": "들판"})
print(response)

Colors.GREEN


## OutputFixingParser
* 출력 파싱 과정에서 발생할 수 있는 오류를 자동으로 수정하는 기능을 제공
* PydanticOutputParser 를 래핑하고, 이 파서가 처리할 수 없는 형식의 출력이나 오류를 반환할 경우, 추가적인 LLM 호출을 통해 오류를 수정하도록 설계

In [23]:
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List

class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(description="list of films they starred in")

actor_query = "Generate the filmography of the actor random famous actor."
parser = PydanticOutputParser(pydantic_object=Actor)

In [24]:
misformatted = "{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}"
parser.parse(misformatted)

OutputParserException: Invalid json output: {'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}

In [25]:
from langchain.output_parsers import OutputFixingParser

new_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI(model="gpt-4o"))
actor = new_parser.parse(misformatted)
actor

Actor(name='Tom Hanks', film_names=['Forrest Gump'])